## Google COVID-19 Community Mobility Reports  
- 코로나 19 확산 방지 정책에 따라 각 국가의 지역사회 이동성 보고서
- 소매점/여가시설/식료품점/약국/공원/대중교통/직장/주거지 등의 카테고리
- https://www.google.com/covid19/mobility/

In [1]:
import pandas as pd
df = pd.read_csv('Global_Mobility_Report.csv')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country_region_code,country_region,sub_region_1,sub_region_2,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682266 entries, 0 to 682265
Data columns (total 13 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   country_region_code                                 681092 non-null  object 
 1   country_region                                      682266 non-null  object 
 2   sub_region_1                                        662016 non-null  object 
 3   sub_region_2                                        393908 non-null  object 
 4   iso_3166_2_code                                     247928 non-null  object 
 5   census_fips_code                                    394059 non-null  float64
 6   date                                                682266 non-null  object 
 7   retail_and_recreation_percent_change_from_baseline  519327 non-null  float64
 8   grocery_and_pharmacy_percent_change_from_baseline   495645 non-n

In [3]:
# 원하는 컬럼 가져오기
df_grouped = df.loc[:, ['date',
'country_region',
'retail_and_recreation_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'parks_percent_change_from_baseline',
'transit_stations_percent_change_from_baseline',
'workplaces_percent_change_from_baseline',
'residential_percent_change_from_baseline']]

In [4]:
# 컬럼 확인
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682266 entries, 0 to 682265
Data columns (total 8 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   date                                                682266 non-null  object 
 1   country_region                                      682266 non-null  object 
 2   retail_and_recreation_percent_change_from_baseline  519327 non-null  float64
 3   grocery_and_pharmacy_percent_change_from_baseline   495645 non-null  float64
 4   parks_percent_change_from_baseline                  308407 non-null  float64
 5   transit_stations_percent_change_from_baseline       362722 non-null  float64
 6   workplaces_percent_change_from_baseline             667223 non-null  float64
 7   residential_percent_change_from_baseline            379408 non-null  float64
dtypes: float64(6), object(2)
memory usage: 41.6+ MB


In [5]:
# columns Rename
df_grouped.columns = [
    'Date',
    'Country',
    'Retail',
    'Grocery',
    'Park',
    'Station',
    'Workplace',
    'Residential'
]

In [6]:
# 결측값 확인
df_grouped.isnull().sum()

Date                0
Country             0
Retail         162939
Grocery        186621
Park           373859
Station        319544
Workplace       15043
Residential    302858
dtype: int64

In [7]:
# 결측치 행제거
df_grouped.dropna(axis=0, inplace=True)

In [8]:
df_grouped.head()

,Date,Country,Retail,Grocery,Park,Station,Workplace,Residential
0,2020-02-15,United Arab Emirates,0.0,4.0,5.0,0.0,2.0,1.0
1,2020-02-16,United Arab Emirates,1.0,4.0,4.0,1.0,2.0,1.0
2,2020-02-17,United Arab Emirates,-1.0,1.0,5.0,1.0,2.0,1.0
3,2020-02-18,United Arab Emirates,-2.0,1.0,5.0,0.0,2.0,1.0
4,2020-02-19,United Arab Emirates,-2.0,0.0,4.0,-1.0,2.0,1.0


In [9]:
# 날짜 확인
df_grouped['Date']

0         2020-02-15
1         2020-02-16
2         2020-02-17
3         2020-02-18
4         2020-02-19
             ...    
681259    2020-07-10
681260    2020-07-11
681261    2020-07-12
681262    2020-07-13
681263    2020-07-14
Name: Date, Length: 245289, dtype: object

In [10]:
# 날짜 재정렬 
df_grouped.sort_values(by='Date', inplace=True)
df_grouped['Date']

0         2020-02-15
148216    2020-02-15
491252    2020-02-15
147914    2020-02-15
491403    2020-02-15
             ...    
120805    2020-07-14
534299    2020-07-14
120654    2020-07-14
529721    2020-07-14
681263    2020-07-14
Name: Date, Length: 245289, dtype: object

In [11]:
# 시각화 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%config InlineBackend.figure_format = 'retina' # 그래프의 글자 선명하게
!apt -qq -y install fonts-nanum  # 나눔 글자체 설치
#나눔고딕 글자체의 전체 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
#글자체의 경로와 글자 크기 설정
font = fm.FontProperties(fname=fontpath, size=9)
#폰트 이름 설정
plt.rc('font', family='NanumBarunGothic') 
#폰트 업데이트
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [12]:
import numpy as np 

# 국가를 func에 대입하면 mobility 시각화 함수 정의
def Mobil_plot(nations):
    x = np.arange(len(df_grouped['Date']))
    plt.style.use('ggplot')
    plt.figure(figsize=(15,8))
    plt.title(nations + ' Community Movement caused by COVID-19')
    # 날짜 groupby, 공공장소 카테고리별 시각화
    plt.plot(df_grouped[df_grouped['Country']==nations].groupby(['Date'])['Retail'].mean(),'-o',label='Retail and leisure facilities')
    plt.plot(df_grouped[df_grouped['Country']==nations].groupby(['Date'])['Grocery'].mean(), '-o', label='Grocery store and pharmacy', color='green')
    plt.plot(df_grouped[df_grouped['Country']==nations].groupby(['Date'])['Park'].mean(), '-o', label='Park', color='blue')
    plt.plot(df_grouped[df_grouped['Country']==nations].groupby(['Date'])['Station'].mean(), '-o', label='Public transport station', color='orange')
    plt.plot(df_grouped[df_grouped['Country']==nations].groupby(['Date'])['Workplace'].mean(), '-o', label='Workplace', color='gray')
    plt.plot(df_grouped[df_grouped['Country']==nations].groupby(['Date'])['Residential'].mean(), '-o', label='Residential', color='magenta')
    plt.legend()

    # 국가별 사회정책
    if nations == 'Russia':
        rdatelist = [
            '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-20',
            '2020-03-28', '2020-03-30', '2020-03-31', '2020-04-02'
        ] # 러시아 휴교령 날짜
        for i, rdate in enumerate(rdatelist):
            plt.axvline(x=rdate, color='g', linestyle=':', linewidth=3, alpha=0.6)    
        rsocial = ['2020-03-29', '2020-03-31', '2020-04-05', '2020-05-09']
        for r, rdate in enumerate(rsocial):
            plt.axvline(x=rdate, color='b', linestyle=':', linewidth=6, alpha=0.6)
        plt.savefig('mobil_RU.png', dpi=300)
        
    if nations == 'United States':
        datelist = [
            '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-23', '2020-03-24', '2020-03-25',
            '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03',
            '2020-04-13', '2020-04-14'] # 미국 휴교령
        social_day = '2020-04-01'
        for i, date in enumerate(datelist):
            plt.axvline(x=date, color='g', linestyle=':', linewidth=3, alpha=0.6)

        plt.axvline(x=social_day, color='b', linestyle=':', linewidth=6, alpha=0.6)
        plt.savefig('mobil_US.png', dpi=300)

    if nations == 'Spain':
        plt.axvline(x='2020-03-11', color='g', linestyle=':', linewidth=3, alpha=0.6) # 스페인 휴교령 날짜.
        plt.axvline(x='2020-05-20', color='b', linestyle=':', linewidth=6, alpha=0.6) # 사회적거리두기
        plt.savefig('mobil_ES.png', dpi=300)

    if nations == 'Brazil':
        bdatelist = [
            '2020-03-04','2020-03-13','2020-03-16','2020-03-18',
            '2020-03-19','2020-03-20','2020-03-23','2020-04-01','2020-05-10']# 브라질 휴교령
        bsocial = ['2020-03-20','2020-04-01'] # 브라질 사회적거리두기
        for i, bdate in enumerate(bdatelist):
            plt.axvline(x=bdate, color='g', linestyle=':', linewidth=3, alpha=0.6)
        for i, bso in enumerate(bsocial):
            plt.axvline(x=bso, color='b', linestyle=':', linewidth=6, alpha=0.6)
        plt.savefig('mobil_BZ.png', dpi=300)

    if nations == 'United Kingdom':
        udatelist = [
            '2020-02-27', '2020-03-20', '2020-03-23'
        ] # 휴교령 날짜
        for i, udate in enumerate(udatelist):
            plt.axvline(x=udate, color='g', linestyle=':', linewidth=3,alpha=0.6)
        plt.axvline(x='2020-03-12', color='b', linestyle=':', linewidth=6,alpha=0.6)
        plt.savefig('mobil_UK.png', dpi=300)

    if nations == 'South Korea':
      kdate = '2020-03-02' # 휴교령
      ksocial = '2020-02-22' # 사회적거리두기
      plt.axvline(x=kdate, color='g', linestyle=':', linewidth=3,alpha=0.6)
      plt.axvline(x=ksocial, color='b', linestyle=':', linewidth=6,alpha=0.6)
      plt.savefig('mobil_KR.png', dpi=300)

    plt.legend()
    plt.show()

In [13]:
Mobil_plot('United States')
Mobil_plot('Brazil')
Mobil_plot('Russia')
Mobil_plot('United Kingdom')
Mobil_plot('Spain')
Mobil_plot('South Korea')

Output hidden; open in https://colab.research.google.com to view.